In [1]:
import torch

In [2]:
x = torch.tensor(
    [[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]], dtype=torch.float32
)
y = torch.tensor([460, 232, 178], dtype=torch.float32)
w = torch.zeros(x.shape[1], requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [3]:

def normalize(x):
    max = torch.max(x,dim=0).values
    min = torch.min(x,dim=0).values
    mean = torch.mean(x,dim=0)
    return (x - mean)/(max - min)

In [4]:
x = normalize(x)
x

tensor([[ 0.5165,  0.5556, -0.3333,  0.5000],
        [-0.0330, -0.1111,  0.6667,  0.0000],
        [-0.4835, -0.4444, -0.3333, -0.5000]])

In [5]:
def loss():
    loss = 0.0
    for i in range(x.shape[0]):
        loss += torch.square((torch.dot(x[i], w) + b) - y[i])
    return loss / (2 * x.shape[0])

In [6]:
l = 5.0e-7

In [7]:
%%time
for i in range(1000):
    loss().backward()
    with torch.no_grad():
        w -= l * w.grad
        b -= l * b.grad
        w.grad.zero_()
        b.grad.zero_()

CPU times: user 170 ms, sys: 13.9 ms, total: 184 ms
Wall time: 183 ms


In [8]:
loss()

tensor(49472.2656, grad_fn=<DivBackward0>)

In [9]:
w

tensor([ 0.0240,  0.0251, -0.0097,  0.0235], requires_grad=True)

In [10]:
def predict(x, y, w):
    for i in range(x.shape[0]):
        pred = torch.dot(x[i], w)
        print(f"Predicted {pred:0.2f}    Expected {y[i]}  Diff {torch.abs(pred - y[i])}")

In [11]:
predict(x, y, w)

Predicted 0.04    Expected 460.0  Diff 459.9587097167969
Predicted -0.01    Expected 232.0  Diff 232.01002502441406
Predicted -0.03    Expected 178.0  Diff 178.03128051757812


In [3]:
class LinearRegression:
    def __init__(self,t_in,t_out):
        self._in = t_in
        self._out = t_out
        self.w = torch.zeros(t_in.shape[1], requires_grad=True)
        self.b = torch.tensor(0.0, requires_grad=True)

    def normalize(self):
        max = torch.max(self._in,dim=0).values
        min = torch.min(self._in,dim=0).values
        mean = torch.mean(self._in,dim=0)
        self._in = (self._in - mean)/(max - min)

    @property
    def _loss(self):
        loss = 0.0
        for i in range(self._in.shape[0]):
            loss += torch.square((torch.dot(self._in[i], w) + b) - self._out[i])
        return loss / (2 * self._in.shape[0])
    
    def train(self,epochs,l_rate):
        for _ in range(epochs):
            self._loss.backward()
            with torch.no_grad():
                self.w -= l_rate * self.w.grad
                self.b -= l_rate * self.b.grad
                self.w.grad.zero_()
                self.b.grad.zero_()

    def predict(x,y):
        for i in range(x.shape[0]):
            pred = torch.dot(x[i], w)
            print(f"Predicted {pred:0.2f}    Expected {y[i]}  Diff {torch.abs(pred - y[i])}")

In [4]:
x

tensor([[2.1040e+03, 5.0000e+00, 1.0000e+00, 4.5000e+01],
        [1.4160e+03, 3.0000e+00, 2.0000e+00, 4.0000e+01],
        [8.5200e+02, 2.0000e+00, 1.0000e+00, 3.5000e+01]])

In [5]:
model = LinearRegression(x,y)

In [6]:
model._in

tensor([[2.1040e+03, 5.0000e+00, 1.0000e+00, 4.5000e+01],
        [1.4160e+03, 3.0000e+00, 2.0000e+00, 4.0000e+01],
        [8.5200e+02, 2.0000e+00, 1.0000e+00, 3.5000e+01]])

In [7]:
model.normalize()

In [8]:
model._in

tensor([[ 0.5165,  0.5556, -0.3333,  0.5000],
        [-0.0330, -0.1111,  0.6667,  0.0000],
        [-0.4835, -0.4444, -0.3333, -0.5000]])

In [11]:
model._loss.backward()

In [15]:
model.w.grad